In [25]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

In [26]:
def extract_2d_array(tensor_str):
    
    tensor_str = tensor_str.replace('tensor(','').replace('grad_fn=<TanhBackward0>)', '').replace(' ', '').replace('\n', '')
    tensor_str = tensor_str[:len(tensor_str)-1]
    numbers = eval(tensor_str)
    
    array = np.array(numbers).reshape((-1, 768))  
    # print(array)
    
    return array

def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False


In [27]:
def constructFinalDataframe(path, bigger_path):

    files = glob.glob(path)
    dfs = []
    for file in files:
        df = pd.read_csv(file)
        df = df.drop(columns=["Unnamed: 0", "Unnamed: 0.1", 'Unnamed: 0.2'], errors="ignore")
        dfs.append(df)
    
    df = pd.concat(dfs, ignore_index=True)
    df_big = pd.read_csv(bigger_path)
    
    combinations = set(df[['name', 'sub_category']].apply(tuple, axis=1))

    # Filter rows in the larger DataFrame that match the combinations in the smaller DataFrame
    filtered_df = df_big[df_big.apply(lambda row: (row['name'], row['sub_category']) in combinations, axis=1)]
    df_unique = filtered_df.drop_duplicates(subset='name', keep='first')

    print(df_unique.shape)
    df_unique.to_csv("dataset_truncated.csv", index=False)

In [28]:
constructFinalDataframe("./final_csvs/*/*.csv", "dataset.csv")

(59017, 10)


In [29]:
df = pd.read_csv("./dataset_truncated.csv")

df = df[df['ratings'].apply(is_float) & df['no_of_ratings'].apply(is_float)]

df['ratings'] = df['ratings'].astype('float64')
df['no_of_ratings'] = df['no_of_ratings'].astype('int')

grouped = df.groupby(by=['sub_category'])
ratings_mean = grouped['ratings'].mean()
print(ratings_mean)
number_mean = grouped['no_of_ratings'].mean()
print(number_mean)

sub_category
Air Conditioners                            3.787563
All Appliances                              3.896424
All Car & Motorbike Products                3.830093
All Electronics                             4.015099
All Exercise & Fitness                      3.878122
All Grocery & Gourmet Foods                 4.155153
All Home & Kitchen                          4.045725
All Pet Supplies                            3.999612
All Sports, Fitness & Outdoors              3.865906
Amazon Fashion                              3.977739
Household Supplies                          3.952387
Indoor Lighting                             3.894112
Innerwear                                   3.657096
Kids' Watches                               3.536500
Lab & Scientific                            3.849910
Lingerie & Nightwear                        3.752994
Musical Instruments & Professional Audio    3.918438
Shirts                                      3.789948
Snack Foods                      

In [30]:
# Group by 'sub_category' and calculate the 1/4th quartile for 'no_of_ratings'
quartile_1_no = grouped['no_of_ratings'].quantile(0.25)
# Print the 1/4th quartile values
print(quartile_1_no)

sub_category
Air Conditioners                             3.0
All Appliances                              15.0
All Car & Motorbike Products                12.0
All Electronics                             64.0
All Exercise & Fitness                       9.0
All Grocery & Gourmet Foods                 81.5
All Home & Kitchen                          48.0
All Pet Supplies                            12.0
All Sports, Fitness & Outdoors              18.0
Amazon Fashion                              39.0
Household Supplies                           4.0
Indoor Lighting                              6.0
Innerwear                                    1.0
Kids' Watches                                1.0
Lab & Scientific                             2.0
Lingerie & Nightwear                         4.0
Musical Instruments & Professional Audio     3.0
Shirts                                       3.0
Snack Foods                                  7.0
Sports Shoes                                 4.0
Strengt

In [31]:
# Group by 'sub_category' and calculate the 1/4th quartile for 'no_of_ratings'
quartile_1_rating = grouped['ratings'].quantile(0.75)
# Print the 1/4th quartile values
print(quartile_1_rating)

sub_category
Air Conditioners                            4.20
All Appliances                              4.20
All Car & Motorbike Products                4.20
All Electronics                             4.30
All Exercise & Fitness                      4.20
All Grocery & Gourmet Foods                 4.30
All Home & Kitchen                          4.30
All Pet Supplies                            4.35
All Sports, Fitness & Outdoors              4.10
Amazon Fashion                              4.20
Household Supplies                          4.50
Indoor Lighting                             4.30
Innerwear                                   4.50
Kids' Watches                               4.10
Lab & Scientific                            4.60
Lingerie & Nightwear                        4.10
Musical Instruments & Professional Audio    4.50
Shirts                                      4.10
Snack Foods                                 4.30
Sports Shoes                                4.10
Strengt

In [32]:
# # Create scatter plots for each sub_category
# for sub_category, group in grouped:
#     plt.figure(figsize=(8, 6))
#     plt.scatter(group['no_of_ratings'], group['ratings'], alpha=0.6)
#     plt.title(f'Scatter Plot for {sub_category}')
#     plt.xlabel('Number of Ratings')
#     plt.ylabel('Ratings')
#     plt.grid(True)
#     plt.show()

In [33]:
def apply_thresholds(row):
    category = row['sub_category']
    return (row['no_of_ratings'] < quartile_1_no[category]) and (row['ratings'] > quartile_1_rating[category])

df_filtered = df[df.apply(apply_thresholds, axis=1)]
print(df_filtered.shape)

(4174, 10)


In [34]:
df_filtered.drop(columns=['Unnamed: 0']).to_csv("Hidden products.csv", index=False)